In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-15 20:41:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-11-15 20:41:38 (10.6 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.types import *  
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# df = spark.read.option('header', 'true').csv(SparkFiles.get(".csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
# df.show(10)


df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

 

df = df.withColumn("customer_id",df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",df["review_date"].cast(DateType()))

df.printSchema()
# df.show(10)


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Count the number of the reviews 
df.count()

1785997

In [ ]:
# Drop duplicates and incomplete rows 
drop_df = df.dropna()
print(drop_df.count())

clean_df = drop_df.dropDuplicates()
print(clean_df.count())

1785886
1785886


In [ ]:
# Check the schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#Transform DataFrame 
# Create a new DataFrame for review info
review_id_table= clean_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005PB41SDWGY|    1246208|B00HN9LS48|     385183836| 2015-02-08|
| R100AUCA9G6HH|   40693899|B002KSPKRS|      44443896| 2009-10-07|
|R10182UTLL75O0|   37307165|B00130387I|     954736702| 2008-04-14|
|R102L60DYWC7BT|   11494390|B004MKN3XU|     171729072| 2011-08-23|
|R102LT5EV9KD0T|    2391530|B00CMQTUSS|     636573779| 2014-06-26|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
products_df = clean_df.select(["product_id", "product_title"])
products = products_df.dropDuplicates(["product_id"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
+----------+--------------------+
only showing top 5 rows



In [ ]:
customer_count = clean_df.select(["customer_id"]).count()
customer_count

1785886

In [ ]:
# Create a new DataFrame for customers
customer_counts = clean_df.groupBy('customer_id').count()
customers = customer_counts.withColumnRenamed('count', 'customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35350046|             2|
|   44433869|             1|
|      51123|             1|
|    9663853|             2|
|   48707250|             1|
|   44445916|             1|
|    1689312|             1|
|   49576952|             3|
|   48515527|             8|
|   15372644|             9|
|   51115911|             2|
|   33598770|             3|
|   50553272|             1|
|    7455053|            50|
|   41457095|             9|
|    9964795|             1|
|   12286053|             1|
|   33217146|             1|
|    6367756|            65|
|    2613933|             5|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005PB41SDWGY|          4|            0|          0|   N|
| R100AUCA9G6HH|          4|            0|          0|   N|
|R10182UTLL75O0|          3|            3|          4|   N|
|R102L60DYWC7BT|          3|            9|         10|   N|
|R102LT5EV9KD0T|          4|            0|          0|   N|
|R102OX47XASUO7|          1|            0|          0|   N|
|R103F2KVCA5L0I|          1|           11|         14|   N|
|R103TA6UYAME81|          3|            0|          0|   N|
|R1045Q4149T2GG|          4|            0|          0|   N|
|R104J4O0ND37GE|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://:5432/postgres"
config = {"user": " ",
          "password": " ",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame table to RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)